In [ ]:
import evalseg
evalseg.reload()
root_data='../datasets'
# dataset_name='Task002_Pancreas'
# case='370.nii'
# ax,frame='z',63
# preds_conf={
#     'p1': 6,
#     'p2': {'x':np.s_[:],'y':np.s_[0:139]},
#     'p3': {'x':np.s_[:],'y':np.s_[-314:-1]},
# }
dataset_name='Task001_LiverTumor'
case='394.nii'
ax,frame='z',126
preds_conf={
    'p1': 5,
    'p2': {'x':np.s_[-257:-1],'y':np.s_[:]},
    'p3': {'x':np.s_[0:234],'y':np.s_[:]}
}
dataset = evalseg.io.NibDataset(f'{root_data}/{dataset_name}')


ct,_=dataset.get_CT(case)
gt,gt_voxelsize=dataset.get_groundtruth(case)

gt, voxelsize = evalseg.geometry.slice(gt, gt_voxelsize, ax,[frame])
ct, _ = evalseg.geometry.slice(ct, None, ax,[frame])
mme = evalseg.metrics.MultiClassMetric(evalseg.metrics.MME,dataset.num_labels)
mme.set_reference(gt, voxelsize)



In [ ]:
import evalseg
evalseg.reload()
# def test(i):
#     print(i)
#     return i*i
[x for x in evalseg.common.parallel_runner(evalseg.metrics.test,range(10))]

In [ ]:
import multiprocessing
multiprocessing.cpu_count()
pool = multiprocessing.Pool(8, maxtasksperchild=20)
# if __name__ == '__main__':

result = pool.imap(evalseg.metrics.test, range(10))
print([r for r in result])

In [ ]:
from multiprocessing import Pool

def f(x):
    return x**2

if __name__ == '__main__':
    pool = Pool(4)
    for res in pool.map(f,range(20)):
        print(res)


In [ ]:
# preds={d:dataset.get(d, case)[0]for d in ['GroundTruth']}
# preds={d:evalseg.geometry.slice(preds[d], None, ax, [frame])[0]for d in preds}
preds_conf={
    'p1': 5,
    'p2': {'x':np.s_[-257:-1],'y':np.s_[:]},
    'p3': {'x':np.s_[0:234],'y':np.s_[:]}
}

pred=gt
for i in range(preds_conf['p1']):
    border=evalseg.geometry.find_binary_boundary(pred,mode='thick')
    pred=pred&~border

pred2=gt.copy()

pred2[preds_conf['p2']['x'],preds_conf['p2']['y']]=0


pred3=gt.copy()
pred3[preds_conf['p3']['x'],preds_conf['p3']['y']]=0


preds={'p1':pred,'p2':pred2,'p3':pred3}
evalseg.reload()
ctlbl = f"{dataset_name.split('_')[1]} {case} {ax}={frame}"
evalseg.ui.multi_plot_2d(ct, gt, preds, spacing=voxelsize, ctlbl= ctlbl, z_titles= [ctlbl],col=6)
res = mme.evaluate_multi(preds,parallel=0)
evalseg.ui.plot_metric_multi(res, name='',show_table=1)


In [ ]:
import nibabel as nib
import numpy as np

import os
data=nib.load(f'C:/Users/Ali/Documents/datasets/All-GT - Copy/{dataset_name}/GroundTruth/{case}.gz')
new_gt = nib.Nifti1Image(gt, data.affine, data.header)
nib.save(new_gt,f'{root_data}/TaskA_Example/GroundTruth/{case}.gz')

new_ct = nib.Nifti1Image(ct, data.affine, data.header)
nib.save(new_ct,f'{root_data}/TaskA_Example/CT/{case}.gz')

for pred in preds:
    dir=f'{root_data}/TaskA_Example/Predictions/{pred}/'
    os.makedirs(dir,exist_ok=True)
    new_pred = nib.Nifti1Image(preds[pred], data.affine, data.header)
    nib.save(new_pred,f'{dir}/{case}.gz')